### Text-to-image LoRA 

In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch

base_model_id = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = StableDiffusionXLPipeline.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

lora_model_path = "./sdxl-lora-minhwa-style" # set your LoRA model path
pipe.load_lora_weights(lora_model_path)

prompt = (
    "Korean traditional minhwa painting, "
    "two person, "
    "a men wearing Korean hat, gat, blue clothes, "
    "a men wearing blue clothes, "
    "with a background of green mountain landscape."
) # set your prompt
image = pipe(prompt=prompt, num_inference_steps=30).images[0]

image.save("minhwa_test_1.png")

### ver2

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline

# --- 1. Base 모델 및 Refiner 모델 불러오기 ---
base_model_id = "stabilityai/stable-diffusion-xl-base-1.0"
refiner_model_id = "stabilityai/stable-diffusion-xl-refiner-1.0"
device = "cuda"

# Base 모델 로드
pipe = StableDiffusionXLPipeline.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safensors=True
).to(device)

# Refiner 모델 로드
refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    refiner_model_id,
    text_encoder_2=pipe.text_encoder_2,
    vae=pipe.vae,
    torch_dtype=torch.float16,
    use_safensors=True,
    variant="fp16",
).to(device)

# --- 2. 학습된 LoRA 가중치 적용 ---
lora_model_path = "./sdxl-lora-minhwa-style-2"
pipe.load_lora_weights(lora_model_path)
# Refiner에도 동일한 LoRA를 적용할 수 있습니다 (선택사항, 스타일에 따라 효과가 다름)
# refiner.load_lora_weights(lora_model_path)


# --- 3. 프롬프트 및 고급 설정 ---
prompt = (
    "masterpiece, best quality, "
    "Korean traditional minhwa painting, "
    "two person, "
    "a man wearing Korean hat, gat, elegant blue clothes, "
    "a woman wearing traditional hanbok, "
    "with a beautiful background of green mountain landscape and pine trees."
)

negative_prompt = (
    "blurry, low quality, worst quality, ugly, deformed, distorted, "
    "watermark, signature, text, jpeg artifacts, poorly drawn"
)

# Refiner를 사용할 때 Base 모델이 어느 정도까지만 디노이징할지 설정
# 0.8은 80% 지점까지만 Base가 작업하고 나머지 20%는 Refiner에게 넘긴다는 의미
high_noise_frac = 0.8

# --- 4. 이미지 생성 (Base -> Refiner) ---
print("이미지 생성을 시작합니다 (Base -> Refiner)...")

# Base 모델 실행, output_type="latent"로 설정하여 Refiner에 넘길 잠재 공간 이미지 생성
latents = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    width=1216,
    height=832,
    num_inference_steps=40,
    guidance_scale=8.0,
    denoising_end=high_noise_frac, # Base 모델이 작업을 멈출 지점
    output_type="latent",
).images[0]

# Refiner 모델 실행, Base에서 받은 latents를 image 인자로 넘김
image = refiner(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=40,
    guidance_scale=8.0,
    denoising_start=high_noise_frac, # Refiner 모델이 작업을 시작할 지점
    image=latents,
).images[0]


# --- 5. 이미지 저장 ---
image.save("minhwa_test_high_quality.png")
print("고품질 이미지가 'minhwa_test_high_quality.png'으로 저장되었습니다.")

## Add Textual Inversion

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline

# --- 1. 모델 및 가중치 경로 설정 ---
base_model_id = "stabilityai/stable-diffusion-xl-base-1.0"
refiner_model_id = "stabilityai/stable-diffusion-xl-refiner-1.0"
lora_model_path = "./sdxl-lora-minhwa-style-2"
textual_inversion_path = "./textual_inversion_korean_house"
device = "cuda"

# --- 2. 베이스 모델 로드 ---
pipe = StableDiffusionXLPipeline.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safensors=True
).to(device)

# --- 3. 학습된 LoRA 가중치 적용 ---
pipe.load_lora_weights(lora_model_path)
print("LoRA 가중치를 적용했습니다.")

# --- 4. 학습된 Textual Inversion 임베딩 적용 ---
pipe.load_textual_inversion(textual_inversion_path)
print("Textual Inversion 임베딩을 적용했습니다.")

# --- 5. Refiner 모델 로드 ---
refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    refiner_model_id,
    text_encoder_2=pipe.text_encoder_2,
    vae=pipe.vae,
    torch_dtype=torch.float16,
    use_safensors=True,
    variant="fp16",
).to(device)
print("Refiner 모델을 로드했습니다.")


# --- 6. 프롬프트 및 고급 설정 ---
# LoRA 스타일과 Textual Inversion placeholder token(<korean-house>)을 모두 사용
prompt = (
    "masterpiece, best quality, "
    "Korean traditional minhwa painting, "
    "<korean-house>." # <-- 학습한 placeholder token 사용
)

negative_prompt = (
    "blurry, low quality, worst quality, ugly, deformed, distorted, "
    "watermark, signature, text, jpeg artifacts, poorly drawn"
)

# Refiner를 사용할 때 Base 모델이 어느 정도까지만 디노이징할지 설정
# 0.8은 80% 지점까지만 Base가 작업하고 나머지 20%는 Refiner에게 넘긴다는 의미
high_noise_frac = 0.8


# --- 7. 이미지 생성 (Base -> Refiner) ---
print("이미지 생성을 시작합니다 (Base -> Refiner)...")

# Base 모델 실행, output_type="latent"로 설정하여 Refiner에 넘길 잠재 공간 이미지 생성
latents = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    width=1024,
    height=1024,
    num_inference_steps=40,
    guidance_scale=9.0,
    denoising_end=high_noise_frac, # Base 모델이 작업을 멈출 지점
    output_type="latent",
).images

# Refiner 모델 실행, Base에서 받은 latents를 image 인자로 넘김
image = refiner(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=40,
    guidance_scale=8.0,
    denoising_start=high_noise_frac, # Refiner 모델이 작업을 시작할 지점
    image=latents,
).images[0]


# --- 8. 이미지 저장 ---
image.save("minhwa_house_with_lora_ti_and_refiner.png")
print("고품질 이미지가 'minhwa_house_with_lora_ti_and_refiner.png'으로 저장되었습니다.")